#
#
#
###### by Vladislav Ofitserov

In [1]:
import cv2
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import LinearSVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
import lightgbm as lgbm
import face_recognition
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

/Users/ofitserovvladislav/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [112]:
# MODE = "MPI"
protoFile = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose/mpi/pose_iter_160000.caffemodel"
nPoints = 15
# POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]

threshold = 0.1 #
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

### The function below needs only for data collection of motions:
### The result:
#### Pandas dataframe (number_of_videos, 100)
### Algorithm:
#### We will take only each forth frame of video(25 first of them). 
#### For each such frame append 4 features - coordinates of displacement  elbow and whrist relative to the shoulder. We use OpenPose network to find points of  elbow, whrist, shoulder

In [423]:
##if you want to see how сorrect a neural network works, you should uncomment lines starting with 1 character #
def extract_video_features(input_sources):
    n_iters = 25
    all_points = []
    for input_source in input_sources:
        print(input_source)
        cap = cv2.VideoCapture(input_source)
        hasFrame, frame = cap.read()
        
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        n_iter = 0
        all_points_x = np.zeros(3, dtype = float).reshape(3,1)
        all_points_y = np.zeros(3, dtype = float).reshape(3,1)
        #vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))
        for k in range(n_iters):
            hasFrame, frame = cap.read()
            hasFrame, frame = cap.read()
            hasFrame, frame = cap.read()
            hasFrame, frame = cap.read()
            if not hasFrame:
                print("Video is too short, just ",n_iter*4, "/100 frames ")
                break
            frameCopy = np.copy(frame)
            
            

            inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (frameWidth // 8, frameHeight // 8),(0, 0, 0), swapRB=False, crop=False)
            net.setInput(inpBlob)
            output = net.forward()

            H = output.shape[2]
            W = output.shape[3]
                ## Array to store the detected keypoints in 1 frame
            points_x = np.zeros(3, dtype = float).reshape(3,1)
            points_y = np.zeros(3, dtype = float).reshape(3,1)

            for i in [2,3,4]: 
                    ## confidence map of corresponding body's part.
                probMap = output[0, i, :, :]

                    ## Find global maxima of the probMap.
                minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

                    ## Scale the point to fit on the original image

                x = point[0] / W
                y = point[1] / H
                
                #cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                #cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

                points_x[i-2] = x 
                points_y[i-2] = y
                

                ## Draw Skeleton
            #   for pair in POSE_PAIRS:
            #       partA = pair[0]
            #       partB = pair[1]
            #       if points[partA] and points[partB]:
            #           cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
            #           cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            #           cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            all_points_x = np.hstack ([all_points_x, points_x], )
            all_points_y = np.hstack ([all_points_y, points_y])

            n_iter+=1
            
            #cv2.putText(frame, "OpenPose using OpenCV", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 50, 0), 2, lineType=cv2.LINE_AA)
            #cv2.imshow('Output-Keypoints', frameCopy)
            #cv2.imshow('Output-Skeleton', frame)
            #vid_writer.write(frameCopy)
            #vid_writer.release()
 
        all_points_x = all_points_x - all_points_x[0]
        all_points_y = all_points_y - all_points_y[0]
        
        agdf, all_points_x, asdf = np.hsplit(all_points_x, [1,100000])
        dsfgfsdg , all_points_y, asdf = np.hsplit(all_points_y, [1,100000])
        
        agdf, all_points_x, asdf = np.vsplit(all_points_x, [1,100000])
        dsfgfsdg , all_points_y, asdf = np.vsplit(all_points_y, [1,100000])
        
        
        image_vector = np.array(np.hstack ([all_points_x, all_points_y])).reshape(-1)
        all_points.append(image_vector)
    return pd.DataFrame({ 'Coord_points': all_points })

## Dataset collection

### Faces

In [6]:
# Load a sample picture and learn how to recognize it.
# We'll load a first frame of video 
cap = cv2.VideoCapture("/Users/ofitserovvladislav/Downloads/IMG_9796.mp4")
hasFrame, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
shestakov_face_encoding = face_recognition.face_encodings(frame)[0]

# Load a second sample picture and learn how to recognize it.
cap = cv2.VideoCapture("/Users/ofitserovvladislav/Public/OPENpose/OpenPose/Video/Поднятие руки/9.mp4")
hasFrame, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
ofitserov_face_encoding = face_recognition.face_encodings(frame)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    shestakov_face_encoding,
    ofitserov_face_encoding
]
known_face_names = [
    "Andrey Shestacov",
    "Ofitserov Vlad"
]
print('Learned encoding for', len(known_face_encodings), 'images.')

Learned encoding for 2 images.


### Hand gesture

#### I've already executed the cell below, saved all results and uploaded them. (cause running of such sell took about (number of videos * 19 ) seconds

In [464]:
# input_sources_hello_1 = [*links on videos with hello motion*]
# video_features_hello_1_ = extract_video_features(input_sources_hello_1)
# video_features_hello_1_.to_pickle("./hello_1_.pkl")

In [456]:
unpickled_hello_1_ = pd.read_pickle("./hello_1_8.pkl")
unpickled_hello_1_['target'] = "Приветствие"
unpickled_hello_1_.head()

,Coord_points,target
0,"[-0.05882352941176472, -0.11764705882352938, -...",Приветствие
1,"[-0.05882352941176466, -0.05882352941176466, -...",Приветствие
2,"[-0.11764705882352938, -0.05882352941176472, -...",Приветствие
3,"[0.0, -0.05882352941176472, -0.058823529411764...",Приветствие
4,"[0.0, -0.05882352941176472, -0.117647058823529...",Приветствие


#### Let's delete videos which consists of less than 100 frames

In [457]:
norm_video_features_hello_1_ = unpickled_hello_1_
k = 0
for i in range (32):
    if len(unpickled_hello_1_['Coord_points'][i]) < 100:
        print(i)
        norm_video_features_hello_1_ = norm_video_features_hello_1_.drop([i - k]).reset_index(drop=True)
        k += 1



7
17


#### Dataframe making

In [458]:
for i in range(100):
    norm_video_features_hello_1_["{}".format(i+1)] = 999.0
    for j in range(30):
        norm_video_features_hello_1_["{}".format(i+1)][j] = norm_video_features_hello_1_['Coord_points'][j][i]
norm_video_features_hello_1_ = norm_video_features_hello_1_.drop(['Coord_points'], axis = 1)
norm_video_features_hello_1_.head()

,target,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,Приветствие,-0.058824,-0.117647,-0.176471,-0.176471,-0.176471,-0.235294,-0.176471,-0.235294,-0.176471,...,-0.133333,-0.133333,-0.100000,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333
1,Приветствие,-0.058824,-0.058824,-0.058824,-0.058824,-0.117647,-0.117647,-0.117647,-0.176471,-0.176471,...,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333
2,Приветствие,-0.117647,-0.058824,-0.117647,-0.117647,-0.117647,-0.117647,-0.117647,-0.176471,-0.176471,...,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333
3,Приветствие,0.000000,-0.058824,-0.058824,-0.117647,-0.117647,-0.117647,-0.117647,-0.117647,-0.117647,...,-0.066667,-0.066667,-0.033333,-0.033333,-0.033333,-0.066667,-0.033333,-0.033333,-0.066667,-0.066667
4,Приветствие,0.000000,-0.058824,-0.117647,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,-0.117647,...,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.033333


#### The same actions for another gesture


In [463]:
# input_sources_raising_1 = [*links on videos with raising_hand motion*]
# video_features_raising_1 = extract_video_features(input_sources_raising_1)
# video_features_raising_1.to_pickle("./raising_1_.pkl")

In [459]:
unpickled_raising_1_ = pd.read_pickle("./raising_1_8.pkl")
unpickled_raising_1_['target'] = "Поднятие руки"
unpickled_raising_1_.head()

,Coord_points,target
0,"[0.0, 0.0, 0.0, 0.0, -0.05882352941176472, -0....",Поднятие руки
1,"[0.0, 0.0, -0.11764705882352938, -0.1764705882...",Поднятие руки
2,"[-0.1764705882352941, -0.11764705882352938, -0...",Поднятие руки
3,"[0.0, -0.05882352941176472, -0.117647058823529...",Поднятие руки
4,"[-0.17647058823529416, -0.17647058823529416, -...",Поднятие руки


In [460]:
norm_video_features_raising_1_ = unpickled_raising_1_
k = 0
for i in range (18):
    if len(unpickled_raising_1_['Coord_points'][i]) < 100:
        print(i)
        norm_video_features_raising_1_ = norm_video_features_raising_1_.drop([i - k]).reset_index(drop=True)
        k += 1

In [461]:
for i in range(100):
    norm_video_features_raising_1_["{}".format(i+1)] = 999.0
    for j in range(18):
        norm_video_features_raising_1_["{}".format(i+1)][j] = norm_video_features_raising_1_['Coord_points'][j][i]
norm_video_features_raising_1_ = norm_video_features_raising_1_.drop(['Coord_points'], axis = 1)
norm_video_features_raising_1_.head()

,target,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,Поднятие руки,0.000000,0.000000,0.000000,0.000000,-0.058824,-0.176471,-0.176471,-0.176471,-0.176471,...,0.200000,0.200000,0.166667,0.166667,0.100000,0.000000,-0.100000,-0.100000,-0.100000,-0.033333
1,Поднятие руки,0.000000,0.000000,-0.117647,-0.176471,-0.117647,-0.117647,-0.117647,-0.117647,-0.117647,...,0.200000,0.100000,0.033333,-0.066667,-0.100000,-0.066667,0.033333,0.133333,0.233333,0.233333
2,Поднятие руки,-0.176471,-0.117647,-0.117647,-0.176471,-0.117647,-0.058824,-0.058824,-0.058824,-0.117647,...,0.233333,0.200000,0.100000,0.000000,-0.066667,-0.033333,0.066667,0.166667,0.233333,0.233333
3,Поднятие руки,0.000000,-0.058824,-0.117647,-0.117647,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,...,0.133333,0.166667,0.200000,0.200000,0.233333,0.233333,0.233333,0.233333,0.233333,0.233333
4,Поднятие руки,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,...,0.200000,0.166667,0.133333,0.100000,0.066667,0.033333,0.033333,0.000000,-0.033333,-0.033333


#### Union of all dataframes in one

In [462]:
norm_video_features = norm_video_features_hello_1_.append(norm_video_features_raising_1_,ignore_index=True)
norm_video_features.head()

,target,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,Приветствие,-0.058824,-0.117647,-0.176471,-0.176471,-0.176471,-0.235294,-0.176471,-0.235294,-0.176471,...,-0.133333,-0.133333,-0.100000,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333,-0.133333
1,Приветствие,-0.058824,-0.058824,-0.058824,-0.058824,-0.117647,-0.117647,-0.117647,-0.176471,-0.176471,...,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333
2,Приветствие,-0.117647,-0.058824,-0.117647,-0.117647,-0.117647,-0.117647,-0.117647,-0.176471,-0.176471,...,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.033333,-0.033333,-0.033333,-0.033333,-0.033333
3,Приветствие,0.000000,-0.058824,-0.058824,-0.117647,-0.117647,-0.117647,-0.117647,-0.117647,-0.117647,...,-0.066667,-0.066667,-0.033333,-0.033333,-0.033333,-0.066667,-0.033333,-0.033333,-0.066667,-0.066667
4,Приветствие,0.000000,-0.058824,-0.117647,-0.176471,-0.176471,-0.176471,-0.176471,-0.176471,-0.117647,...,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.066667,-0.033333


### Now let's try to find ML model for our case

In [439]:
X_train, X_test, y_train, y_test = train_test_split(
    norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], test_size=0.2, random_state=120)

In [440]:
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3).fit(X_train, y_train) # + 42
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))
# clf.predict(X_test)

Accuracy: 0.91 (+/- 0.21)
[1.         1.         1.         1.         0.66666667 1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ]


In [441]:
clf = LinearSVC(random_state=1, tol=1e-9)
# clf.predict(X_test)
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))

Accuracy: 0.96 (+/- 0.21)
[1.         1.         1.         0.75       1.         1.
 1.         0.66666667 1.         1.         1.         1.
 1.         1.        ]


In [442]:
clf = svm.SVC(gamma='scale', decision_function_shape='ovo', C = 100 , probability=True)
# print(clf.predict(X_test))
# print(clf.predict_proba(X_test))
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))

Accuracy: 0.96 (+/- 0.26)
[1.  1.  1.  0.5 1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]


In [443]:
clf = KNeighborsClassifier(n_neighbors=4).fit(X_train, y_train)
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))
# clf.predict(X_test)

Accuracy: 0.89 (+/- 0.35)
[1.         1.         1.         0.5        1.         0.66666667
 1.         0.66666667 1.         0.66666667 1.         1.
 1.         1.        ]


In [444]:
clf = XGBClassifier().fit(X_train, y_train)
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))
# print(clf.predict(X_test))
# model.predict_proba(X_test)

Accuracy: 0.89 (+/- 0.35)
[1.         1.         1.         0.5        0.66666667 1.
 1.         0.66666667 1.         0.66666667 1.         1.
 1.         1.        ]


In [445]:
clf = AdaBoostClassifier(n_estimators=100).fit(X_train, y_train)
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))
# print(clf.predict(X_test))
# clf.predict_proba(X_test)

Accuracy: 0.93 (+/- 0.33)
[1.         1.         1.         0.75       1.         1.
 0.66666667 0.66666667 1.         1.         1.         1.
 1.         1.        ]


In [446]:
clf = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass'
                          )
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))
# print(clf.predict(X_test))
# model.predict_proba(X_test)

0:	learn: 0.5014795	total: 11.2ms	remaining: 101ms
1:	learn: 0.3340812	total: 13.4ms	remaining: 53.5ms
2:	learn: 0.2171500	total: 14.5ms	remaining: 33.9ms
3:	learn: 0.1743647	total: 15.7ms	remaining: 23.6ms
4:	learn: 0.1553975	total: 16.8ms	remaining: 16.8ms
5:	learn: 0.1316485	total: 17.7ms	remaining: 11.8ms
6:	learn: 0.0897754	total: 19.6ms	remaining: 8.39ms
7:	learn: 0.0803496	total: 21.1ms	remaining: 5.26ms
8:	learn: 0.0665219	total: 22ms	remaining: 2.44ms
9:	learn: 0.0491730	total: 22.8ms	remaining: 0us
0:	learn: 0.5443912	total: 1.03ms	remaining: 9.27ms
1:	learn: 0.3674610	total: 1.99ms	remaining: 7.97ms
2:	learn: 0.2542495	total: 2.88ms	remaining: 6.71ms
3:	learn: 0.2151298	total: 3.7ms	remaining: 5.55ms
4:	learn: 0.1615053	total: 4.65ms	remaining: 4.65ms
5:	learn: 0.1514955	total: 5.63ms	remaining: 3.75ms
6:	learn: 0.1166584	total: 6.47ms	remaining: 2.77ms
7:	learn: 0.0969553	total: 8.21ms	remaining: 2.05ms
8:	learn: 0.0825379	total: 9.45ms	remaining: 1.05ms
9:	learn: 0.0590540

0:	learn: 0.3308793	total: 1.89ms	remaining: 17ms
1:	learn: 0.2825524	total: 3.88ms	remaining: 15.5ms
2:	learn: 0.1994472	total: 5.38ms	remaining: 12.5ms
3:	learn: 0.1620790	total: 7.65ms	remaining: 11.5ms
4:	learn: 0.1067867	total: 8.97ms	remaining: 8.97ms
5:	learn: 0.0792597	total: 9.97ms	remaining: 6.65ms
6:	learn: 0.0543519	total: 10.7ms	remaining: 4.6ms
7:	learn: 0.0462342	total: 11.7ms	remaining: 2.93ms
8:	learn: 0.0387775	total: 12.8ms	remaining: 1.42ms
9:	learn: 0.0321850	total: 13.8ms	remaining: 0us
0:	learn: 0.4711680	total: 753us	remaining: 6.78ms
1:	learn: 0.3071559	total: 1.65ms	remaining: 6.59ms
2:	learn: 0.1995491	total: 2.45ms	remaining: 5.72ms
3:	learn: 0.1875527	total: 3.27ms	remaining: 4.9ms
4:	learn: 0.1456074	total: 4.12ms	remaining: 4.12ms
5:	learn: 0.1132489	total: 4.93ms	remaining: 3.29ms
6:	learn: 0.0781140	total: 5.71ms	remaining: 2.45ms
7:	learn: 0.0569794	total: 6.62ms	remaining: 1.66ms
8:	learn: 0.0431619	total: 7.56ms	remaining: 840us
9:	learn: 0.0348670	t

0:	learn: 0.4893202	total: 752us	remaining: 6.77ms
1:	learn: 0.2851097	total: 2.51ms	remaining: 10.1ms
2:	learn: 0.1986031	total: 4.22ms	remaining: 9.84ms
3:	learn: 0.1579626	total: 5.85ms	remaining: 8.77ms
4:	learn: 0.1070171	total: 7.41ms	remaining: 7.41ms
5:	learn: 0.0828608	total: 9.41ms	remaining: 6.28ms
6:	learn: 0.0605118	total: 11.6ms	remaining: 4.95ms
7:	learn: 0.0463471	total: 12.3ms	remaining: 3.08ms
8:	learn: 0.0385531	total: 13.1ms	remaining: 1.46ms
9:	learn: 0.0339952	total: 14.1ms	remaining: 0us
0:	learn: 0.4506219	total: 750us	remaining: 6.75ms
1:	learn: 0.2501989	total: 1.68ms	remaining: 6.71ms
2:	learn: 0.1814505	total: 2.51ms	remaining: 5.86ms
3:	learn: 0.1515258	total: 3.3ms	remaining: 4.95ms
4:	learn: 0.1139687	total: 4.1ms	remaining: 4.1ms
5:	learn: 0.0965810	total: 4.87ms	remaining: 3.25ms
6:	learn: 0.0654067	total: 5.63ms	remaining: 2.41ms
7:	learn: 0.0495951	total: 6.4ms	remaining: 1.6ms
8:	learn: 0.0422209	total: 7.21ms	remaining: 801us
9:	learn: 0.0365723	tot

In [447]:
clf = GradientBoostingClassifier(n_estimators=100, 
                                    max_features=4, 
                                    max_depth=4, 
                                    random_state=1
                                   )
print("Accuracy: %0.2f (+/- %0.2f)" % 
      (cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).mean(), 
            cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14).std() * 2))
print(cross_val_score(clf, norm_video_features.drop(['target'], axis = 1), norm_video_features["target"], cv=14))
# print(y_test)
# print(clf.predict_proba(X_test))

Accuracy: 0.98 (+/- 0.13)
[1.   1.   1.   0.75 1.   1.   1.   1.   1.   1.   1.   1.   1.   1.  ]


### As we can see GBC shows the most accurate result, but at the same time, this method is quite fast compared to, for example, Ada (which about the same accuracy)
### But for my mind we should lose a little in quality, but to do this process almost instantly => will choose SVC
### ------

# Prediction
### The function below needs for our prediction (it's look like very similar to our first function for data collection):
### The result:
#### Access verdict
### Algorithm:
#### Firstly we take first frame and recognize the man (if he/she is not in the access_table we wouldn't give access for him/her and wouldn't do any other steps)
#### If we "know" this man. We make the same dataframe as in data collection and predict motion. 
#### Then compare the model prediction with motion of this man in our "name-motion table".

In [448]:
##if you want to see how сorrect a neural network works, you should uncomment lines starting with 1 character #
def access_request(input_sources):
    n_iters = 25
    all_points = []
    name = []
    gest = []
    n_of_video = -1
    f = 1
    for input_source in input_sources:
        
        print(input_source)
        cap = cv2.VideoCapture(input_source)
        hasFrame, frame = cap.read()
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        
        n_iter = 0
        all_points_x = np.zeros(3, dtype = float).reshape(3,1)
        all_points_y = np.zeros(3, dtype = float).reshape(3,1)
        #vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))
        for k in range(n_iters):
            
            hasFrame, frame = cap.read()
            hasFrame, frame = cap.read()
            hasFrame, frame = cap.read()
            hasFrame, frame = cap.read()
            if not hasFrame:
                print("Video is too short, just ", n_iter*4, "/100 frames ")
                break
            
            if n_iter == 1:
                unknown_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Find the face and face encoding in the unknown image
                face_locations = face_recognition.face_locations(unknown_image)
                face_encodings = face_recognition.face_encodings(unknown_image, face_locations)
                matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
                face_distances = face_recognition.face_distance(known_face_encodings, face_encodings[0])
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name.append(known_face_names[best_match_index])
                    n_of_video += 1;
                else:
                    print("\n No access! \n\n")
                    f = 0
                    break
          
            
            #frameCopy = np.copy(frame)
            
            
            
            inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (frameWidth // 8, frameHeight // 8),(0, 0, 0), swapRB=False, crop=False)
            net.setInput(inpBlob)
            output = net.forward()

            H = output.shape[2]
            W = output.shape[3]
                ## Array to store the detected keypoints in 1 frame
            points_x = np.zeros(3, dtype = float).reshape(3,1)
            points_y = np.zeros(3, dtype = float).reshape(3,1)
            for i in [2,3,4]: 
                    ## confidence map of corresponding body's part.
                probMap = output[0, i, :, :]
                
                    ## Find global maxima of the probMap.
                minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

                    ## Scale the point to fit on the original image
                x = (point[0]) / W
                y = (point[1]) / H

                #cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                #cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

                points_x[i-2] = x
                points_y[i-2] = (y)

                ## Draw Skeleton
            #   for pair in POSE_PAIRS:
            #       partA = pair[0]
            #       partB = pair[1]
            #       if points[partA] and points[partB]:
            #           cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
            #           cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            #           cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            all_points_x = np.hstack ([all_points_x, points_x])
            all_points_y = np.hstack ([all_points_y, points_y])

            n_iter+=1
            
            #cv2.putText(frame, "OpenPose using OpenCV", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 50, 0), 2, lineType=cv2.LINE_AA)
            #cv2.imshow('Output-Keypoints', frameCopy)
            #cv2.imshow('Output-Skeleton', frame)
            #vid_writer.write(frameCopy)
            #vid_writer.release()
        if (f): 
            all_points_x = all_points_x - all_points_x[0]
            all_points_y = all_points_y - all_points_y[0]

            asdf, all_points_x, asdf = np.hsplit(all_points_x, [1,100000])
            asdf, all_points_y, asdf = np.hsplit(all_points_y, [1,100000])

            asdf, all_points_x, asdf = np.vsplit(all_points_x, [1,100000])
            asdf, all_points_y, asdf = np.vsplit(all_points_y, [1,100000])

            image_vector = np.array(np.hstack ([all_points_x, all_points_y])).reshape(-1)
            norm_video_features_test = pd.DataFrame({'Coord_points': image_vector})
            for i in range(100):
                norm_video_features_test["{}".format(i+1)] = 999
                norm_video_features_test["{}".format(i+1)] = norm_video_features_test['Coord_points'][i]
            norm_video_features_test = norm_video_features_test.drop(['Coord_points'], axis = 1)
            predict = clf.predict_proba(norm_video_features_test)
            if predict[0][0] > 0.7:
                gest.append("Поднятие руки")
            elif predict[0][1] > 0.7:
                gest.append("Приветствие")
            else: 
                gest.append("Другой жест")

            if (gest[n_of_video] == name_motion_table[name[n_of_video]]):
                print("\n Hello, ", name[n_of_video], "! \n\n")
            elif name[n_of_video] in access_df:
                print("\n Please try again,", name[n_of_video], "[", gest[n_of_video], "] \n\n")
        else:
            f = 1


In [449]:
clf = svm.SVC(gamma='scale', 
              decision_function_shape='ovo', 
              C = 100,
              probability=True
             ).fit(norm_video_features.drop(['target'], axis = 1), norm_video_features['target'])

In [450]:
name_motion_table = {'Andrey Shestacov': "Приветствие"}
input_sources_test = ["/Users/ofitserovvladislav/Downloads/IMG_9796.mp4",
                      "/Users/ofitserovvladislav/Downloads/IMG_9797.mp4",
                      "/Users/ofitserovvladislav/Downloads/IMG_9798.mp4"
                     ]

In [451]:
access_request(input_sources_test)

/Users/ofitserovvladislav/Downloads/IMG_9796.mp4

 Hello,  Andrey Shestacov ! 


/Users/ofitserovvladislav/Downloads/IMG_9797.mp4

 Please try again, Andrey Shestacov [ Поднятие руки ] 


/Users/ofitserovvladislav/Downloads/IMG_9798.mp4

 No access! 


